<a href="https://colab.research.google.com/github/VrijKun/CS6910_Assignment_1/blob/main/Assignment_1_DL_ED23D015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.2 MB/s eta 0:00:00


In [2]:
import wandb
wandb.login() #3c21150eb43b007ee446a1ff6e87f640ec7528c4

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
from keras.datasets import fashion_mnist
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Load the Fashion-MNIST dataset
(X_train, Y_train), (X_test, Y_test) = fashion_mnist.load_data()


image_classes = len(np.unique(Y_train)) #to find out number of unique images
image_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot'] #taken from keras documentation

def show_images(image_classe,img_labels,X,Y):
   set_Labels=[]
   set_Images = []

   for i in range(len(X)):
     if len(set_Images)== image_classes:
       break
     if image_labels[Y[i]] not in set_Labels:

      set_Labels.append(image_labels[Y[i]])
      set_Images.append(X[i])

   run=wandb.init(project='DL_Assignment_1',entity='ed23d015',reinit=True)
   run.log({"Sample from each class":list(wandb.Image(img,caption= caption) for img,caption in zip(set_Images,set_Labels))})
   run.finish()

#Getting prints
show_images(image_classes,image_labels,X_test,Y_test)

4422102/4422102 [==============================] - 0s 0us/step


wandb: Currently logged in as: ed23d015. Use `wandb login --relogin` to force relogin


In [ ]:
# The code works for all the optimizers''' This is iterarion 2 lol remove this if other works out
def initialize_parameters(layer_dims):
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
    return parameters

def sigmoid(Z):
    return 1 / (1 + np.exp(-Z))

def relu(Z):
    return np.maximum(0, Z)

def softmax(Z):
    exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
    return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

def forward_propagation(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2
    for l in range(1, L):
        A_prev = A
        Z = np.dot(parameters['W' + str(l)], A_prev) + parameters['b' + str(l)]
        A = relu(Z)
        cache = (A_prev, parameters['W' + str(l)], parameters['b' + str(l)], Z)
        caches.append(cache)
    ZL = np.dot(parameters['W' + str(L)], A) + parameters['b' + str(L)]
    AL = softmax(ZL)
    cache = (A, parameters['W' + str(L)], parameters['b' + str(L)], ZL)
    caches.append(cache)
    return AL, caches

def compute_cost(AL, Y):
    m = Y.shape[1]
    cost = -np.sum(np.multiply(Y, np.log(AL))) / m
    return cost

def backward_propagation(AL, Y, caches):
    grads = {}
    L = len(caches)
    m = AL.shape[1]
    Y = Y.reshape(AL.shape)
    dZL = AL - Y
    A_prev, W, b, Z = caches[L-1]
    grads['dW' + str(L)] = np.dot(dZL, A_prev.T) / m
    grads['db' + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m
    dAL = np.dot(W.T, dZL)
    for l in reversed(range(L-1)):
        A_prev, W, b, Z = caches[l]
        dZ = np.array(dAL, copy=True)
        dZ[Z <= 0] = 0
        grads['dW' + str(l + 1)] = np.dot(dZ, A_prev.T) / m
        grads['db' + str(l + 1)] = np.sum(dZ, axis=1, keepdims=True) / m
        dAL = np.dot(W.T, dZ)
    return grads

def update_parameters(parameters, grads, learning_rate, method="SGD", beta1=0.9, beta2=0.999, epsilon=1e-8):
    L = len(parameters) // 2
    if method == "SGD":
        for l in range(L):
            parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)]
            parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)]
    elif method == "Momentum":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)]
            parameters['W' + str(l+1)] -= learning_rate * v_dW
            parameters['b' + str(l+1)] -= learning_rate * v_db
    elif method == "Nesterov":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)]
            parameters['W' + str(l+1)] -= learning_rate * (v_dW + beta1 * v_dW)
            parameters['b' + str(l+1)] -= learning_rate * (v_db + beta1 * v_db)
    elif method == "RMSprop":
        for l in range(L):
            s_dW = beta2 * grads['dW' + str(l+1)]**2 + (1 - beta2) * (grads['dW' + str(l+1)]**2)
            s_db = beta2 * grads['db' + str(l+1)]**2 + (1 - beta2) * (grads['db' + str(l+1)]**2)
            parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)] / (np.sqrt(s_dW) + epsilon)
            parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)] / (np.sqrt(s_db) + epsilon)
    elif method == "Adam":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)] + (1 - beta1) * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)] + (1 - beta1) * grads['db' + str(l+1)]
            s_dW = beta2 * grads['dW' + str(l+1)]**2 + (1 - beta2) * (grads['dW' + str(l+1)]**2)
            s_db = beta2 * grads['db' + str(l+1)]**2 + (1 - beta2) * (grads['db' + str(l+1)]**2)
            parameters['W' + str(l+1)] -= learning_rate * v_dW / (np.sqrt(s_dW) + epsilon)
            parameters['b' + str(l+1)] -= learning_rate * v_db / (np.sqrt(s_db) + epsilon)
    elif method == "Nadam":
        for l in range(L):
            v_dW = beta1 * grads['dW' + str(l+1)] + (1 - beta1) * grads['dW' + str(l+1)]
            v_db = beta1 * grads['db' + str(l+1)] + (1 - beta1) * grads['db' + str(l+1)]
            s_dW = beta2 * grads['dW' + str(l+1)]**2 + (1 - beta2) * (grads['dW' + str(l+1)]**2)
            s_db = beta2 * grads['db' + str(l+1)]**2 + (1 - beta2) * (grads['db' + str(l+1)]**2)
            v_dW_corrected = v_dW / (1 - beta1)
            v_db_corrected = v_db / (1 - beta1)
            s_dW_corrected = s_dW / (1 - beta2)
            s_db_corrected = s_db / (1 - beta2)
            parameters['W' + str(l+1)] -= learning_rate * v_dW_corrected / (np.sqrt(s_dW_corrected) + epsilon)
            parameters['b' + str(l+1)] -= learning_rate * v_db_corrected / (np.sqrt(s_db_corrected) + epsilon)
    return parameters

def predict(X, parameters):
    AL, _ = forward_propagation(X, parameters)
    return np.argmax(AL, axis=0)

def accuracy(predictions, labels):
    return np.mean(predictions == np.argmax(labels, axis=0))


def to_categorical(labels, num_classes):
    # Initialize an array of zeros with shape (num_classes, num_samples)
    one_hot = np.zeros((num_classes, labels.shape[0]))
    # Set the corresponding index of each sample to 1
    one_hot[labels, np.arange(labels.shape[0])] = 1
    return one_hot.T  # Transpose the array to match the desired shape

# Preprocess the data
train_images = X_train / 255.0
test_images = X_test / 255.0
train_images = train_images.reshape(train_images.shape[0], -1).T
test_images = test_images.reshape(test_images.shape[0], -1).T

# Convert labels to one-hot encoding
train_labels = to_categorical(Y_train, num_classes=10).T
test_labels = to_categorical(Y_test, num_classes=10).T

# Define hyperparameters
layer_dims = [train_images.shape[0], 256,256, 10]
learning_rate = 0.01
num_epochs = 10

# Initialize parameters
parameters = initialize_parameters(layer_dims)

# Train the model with different optimization methods
optimization_methods = ["SGD", "Momentum", "Nesterov", "RMSprop", "Adam", "Nadam"]
for method in optimization_methods:
    print(f"\nTraining with {method} optimizer...")
    parameters = initialize_parameters(layer_dims)  # Re-initialize parameters for each optimization method
    for epoch in range(num_epochs):
        AL, caches = forward_propagation(train_images, parameters)
        cost = compute_cost(AL, train_labels)
        grads = backward_propagation(AL, train_labels, caches)
        parameters = update_parameters(parameters, grads, learning_rate, method=method)
        print(f"Epoch {epoch+1}/{num_epochs}, Cost: {cost:.4f}")

    # Evaluate the model
    train_predictions = predict(train_images, parameters)
    test_predictions = predict(test_images, parameters)
    train_acc = accuracy(train_predictions, train_labels)
    test_acc = accuracy(test_predictions, test_labels)
    print(f"Train Accuracy ({method}): {train_acc:.4f}")
    print(f"Test Accuracy ({method}): {test_acc:.4f}")


In [28]:
#Class is working as well. KEEP IT

class NurlNtwk:
    def __init__(self, input_layer, hidden_layers, output_layer, initialization, activation, loss_function, optimizer, dropout_rate=0):
        self.input_layer = input_layer
        self.hidden_layers = hidden_layers
        self.output_layer = output_layer
        self.initialization = initialization
        self.activation = activation
        self.loss_function = loss_function
        self.optimizer = optimizer
        self.dropout_rate = dropout_rate
        self.parameters = None

    def initialize_parameters(self):
        np.random.seed(1)
        parameters = {}
        layer_dims = [self.input_layer] + self.hidden_layers + [self.output_layer]
        L = len(layer_dims)
        for l in range(1, L):
            if self.initialization == "random":
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
            elif self.initialization == "Xavier":
                parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * np.sqrt(1 / layer_dims[l-1])
            parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        self.parameters = parameters

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def relu(self, Z):
        return np.maximum(0, Z)

    def softmax(self, Z):
        exp_Z = np.exp(Z - np.max(Z, axis=0, keepdims=True))
        return exp_Z / np.sum(exp_Z, axis=0, keepdims=True)

    def forward_propagation(self, X):
        caches = []
        A = X
        L = len(self.parameters) // 2
        for l in range(1, L):
            A_prev = A
            Z = np.dot(self.parameters['W' + str(l)], A_prev) + self.parameters['b' + str(l)]
            if self.activation == "sigmoid":
                A = self.sigmoid(Z)
            elif self.activation == "relu":
                A = self.relu(Z)
            cache = (A_prev, self.parameters['W' + str(l)], self.parameters['b' + str(l)], Z)
            caches.append(cache)
        ZL = np.dot(self.parameters['W' + str(L)], A) + self.parameters['b' + str(L)]
        AL = self.softmax(ZL)
        cache = (A, self.parameters['W' + str(L)], self.parameters['b' + str(L)], ZL)
        caches.append(cache)
        return AL, caches

    def compute_cost(self, AL, Y):
        m = Y.shape[1]
        if self.loss_function == "cross_entropy":
            cost = -np.sum(np.multiply(Y, np.log(AL))) / m
        return cost

    def backward_propagation(self, AL, Y, caches):
        grads = {}
        L = len(caches)
        m = AL.shape[1]
        Y = Y.reshape(AL.shape)
        dZL = AL - Y
        A_prev, W, b, Z = caches[L-1]
        grads['dW' + str(L)] = np.dot(dZL, A_prev.T) / m
        grads['db' + str(L)] = np.sum(dZL, axis=1, keepdims=True) / m
        dAL = np.dot(W.T, dZL)
        for l in reversed(range(L-1)):
            A_prev, W, b, Z = caches[l]
            dZ = np.array(dAL, copy=True)
            dZ[Z <= 0] = 0
            grads['dW' + str(l + 1)] = np.dot(dZ, A_prev.T) / m
            grads['db' + str(l + 1)] = np.sum(dZ, axis=1, keepdims=True) / m
            dAL = np.dot(W.T, dZ)
        return grads

    def initialize_optimizer(self):
        self.velocity = {}
        self.momentum = 0
        self.rmsprop_cache = {}
        self.adam_params = {'m': {}, 'v': {}}
        self.nadam_params = {'m': {}, 'v': {}}

    def update_parameters(self, grads, learning_rate):
        L = len(self.parameters) // 2
        for l in range(L):
            if self.optimizer == "SGD":
                self.parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)]
                self.parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)]
            elif self.optimizer == "Momentum":
                v_dW = 0.9 * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)]
                self.parameters['W' + str(l+1)] -= learning_rate * v_dW
                self.parameters['b' + str(l+1)] -= learning_rate * v_db
            elif self.optimizer == "Nesterov":
                v_dW = 0.9 * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)]
                self.parameters['W' + str(l+1)] -= learning_rate * (v_dW + 0.9 * v_dW)
                self.parameters['b' + str(l+1)] -= learning_rate * (v_db + 0.9 * v_db)
            elif self.optimizer == "RMSprop":
                s_dW = 0.99 * grads['dW' + str(l+1)]**2 + (1 - 0.99) * (grads['dW' + str(l+1)]**2)
                s_db = 0.99 * grads['db' + str(l+1)]**2 + (1 - 0.99) * (grads['db' + str(l+1)]**2)
                self.parameters['W' + str(l+1)] -= learning_rate * grads['dW' + str(l+1)] / (np.sqrt(s_dW) + (1e-8))
                self.parameters['b' + str(l+1)] -= learning_rate * grads['db' + str(l+1)] / (np.sqrt(s_db) + (1e-8))
            elif self.optimizer == "Adam":
                v_dW = 0.9 * grads['dW' + str(l+1)] + (1 - 0.9) * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)] + (1 - 0.9) * grads['db' + str(l+1)]
                s_dW = 0.99 * grads['dW' + str(l+1)]**2 + (1 - 0.99) * (grads['dW' + str(l+1)]**2)
                s_db = 0.99 * grads['db' + str(l+1)]**2 + (1 - 0.99) * (grads['db' + str(l+1)]**2)
                self.parameters['W' + str(l+1)] -= learning_rate * v_dW / (np.sqrt(s_dW) + (1e-8))
                self.parameters['b' + str(l+1)] -= learning_rate * v_db / (np.sqrt(s_db) + (1e-8))
            elif self.optimizer == "Nadam":
                v_dW = 0.9 * grads['dW' + str(l+1)] + (1 - 0.9) * grads['dW' + str(l+1)]
                v_db = 0.9 * grads['db' + str(l+1)] + (1 - 0.9) * grads['db' + str(l+1)]
                s_dW = 0.99 * grads['dW' + str(l+1)]**2 + (1 - 0.99) * (grads['dW' + str(l+1)]**2)
                s_db = 0.99 * grads['db' + str(l+1)]**2 + (1 - 0.99) * (grads['db' + str(l+1)]**2)
                v_dW_corrected = v_dW / (1 - 0.9)
                v_db_corrected = v_db / (1 - 0.9)
                s_dW_corrected = s_dW / (1 - 0.99)
                s_db_corrected = s_db / (1 - 0.99)
                self.parameters['W' + str(l+1)] -= learning_rate * v_dW_corrected / (np.sqrt(s_dW_corrected) + (1e-8))
                self.parameters['b' + str(l+1)] -= learning_rate * v_db_corrected / (np.sqrt(s_db_corrected) + (1e-8))

    def train(self, X_train, Y_train, X_val, Y_val, num_epochs, learning_rate, beta1=0.9, beta2=0.999, epsilon=1e-8):
        train_costs = []
        val_costs = []


        for epoch in range(num_epochs):
                self.initialize_parameters()
                AL, caches = self.forward_propagation(X_train)
                train_cost = self.compute_cost(AL, Y_train)
                train_costs.append(train_cost)
                val_AL, _ = self.forward_propagation(X_val)
                val_cost = self.compute_cost(val_AL, Y_val)
                val_costs.append(val_cost)
                grads = self.backward_propagation(AL, Y_train, caches)

                train_predictions = self.predict(X_train)
                val_predictions = self.predict(X_val)
                train_acc = self.accuracy(train_predictions, Y_train)
                val_acc = self.accuracy(val_predictions, Y_val)

                if self.optimizer == "SGD":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "Momentum":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer== "Nesterov":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "RMSprop":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "Adam":
                    self.update_parameters(grads, learning_rate)
                elif self.optimizer == "Nadam":
                    self.update_parameters(grads, learning_rate)
                print(f"Epoch {epoch+1}/{num_epochs}, Train Cost: {train_cost:.4f}, Val Cost: {val_cost:.4f}, train_acc: {train_acc:.4f}")
        return train_costs, val_costs

    def predict(self, X):
        AL, _ = self.forward_propagation(X)
        return np.argmax(AL, axis=0)

    def accuracy(self, predictions, labels):
        return np.mean(predictions == np.argmax(labels, axis=0))

# Load the Fashion-MNIST dataset
(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

# Preprocess the data
train_images = train_images / 255.0
test_images = test_images / 255.0
train_images = train_images.reshape(train_images.shape[0], -1).T
test_images = test_images.reshape(test_images.shape[0], -1).T

def to_categorical(labels, num_classes):
    # Initialize an array of zeros with shape (num_classes, num_samples)
    one_hot = np.zeros((num_classes, labels.shape[0]))
    # Set the corresponding index of each sample to 1
    one_hot[labels, np.arange(labels.shape[0])] = 1
    return one_hot.T  # Transpose the array to match the desired shape

# Convert labels to one-hot encoding
train_labels = to_categorical(train_labels, num_classes=10).T
test_labels = to_categorical(test_labels, num_classes=10).T

# Split train set into train and validation
val_split = int(train_images.shape[1] * 0.1)
val_images = train_images[:, :val_split]
val_labels = train_labels[:, :val_split]
train_images = train_images[:, val_split:]
train_labels = train_labels[:, val_split:]


# Define hyperparameters
input_layer = train_images.shape[0]
hidden_layers = [256,256]
output_layer = 10
initialization = "Xavier"
activation = "relu"
loss_function = "cross_entropy"
dropout_rate = 0.2
learning_rate = 0.001
num_epochs = 20
optimizer = "Adam"
# Create and train the network
network = NurlNtwk(input_layer, hidden_layers, output_layer, initialization, activation, loss_function, optimizer, dropout_rate)
train_costs, val_costs = network.train(train_images, train_labels, val_images, val_labels, num_epochs, learning_rate)
# Evaluate the model
train_predictions = network.predict(train_images)
val_predictions = network.predict(val_images)
train_acc = network.accuracy(train_predictions, train_labels)
val_acc = network.accuracy(val_predictions, val_labels)
print(f"Train Accuracy: {train_acc:.4f}")
print(f"Validation Accuracy: {val_acc:.4f}")


Epoch 1/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 2/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 3/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 4/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 5/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 6/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 7/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 8/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 9/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 10/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 11/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 12/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 13/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 14/20, Train Cost: 2.3127, Val Cost: 2.3169, train_acc: 0.1289
Epoch 15/20, Train Cost: 2.3127, Val Cost: 

In [ ]:
sweep_config = {
    'method': 'bayes', #grid, random,bayes
    'metric': {
      'name': 'val_accuracy',
      'goal': 'maximize'
    },
    'parameters': {
        'epochs': {
            'values': [10,20,30]
        },
        'learning_rate': {
            'values': [0.001,0.0001]
        },
        'loss_function':{
            'values':['cross_entropy']
        },
        'initilisation':{
            'values':['random','xavier']
        },
        'batch_size':{
            'values':[32,64,128]
        },
        'optimiser': {
            'values': ['gd','mgd','ngd', 'rmsprop','adam','nadam']
        },
        'activation': {
            'values': ['relu','tanh','sigmoid']
        },
        'hidden_layer': {
            'values': [
                       [64,64,64],[128,128,128],[256,256,256],
                       [64,64,64,64],[64,64,64,64,64],
                       [128,128,128,128],[128,128,128,128,128]]
        },

        'weight_decay':{
            'values':[0.0,0.0005]
        },
        'dropout_rate':{
          'values':[0,0.1]
        }

    }
}

sweep_id = wandb.sweep(sweep_config, entity='shreyashgadgil007', project="CS-6910 A1")

def train1():
  steps = 10
  # Default values for hyper-parameters we're going to sweep over
  config_defaults = {
      'epochs': 5,
      'input_layer': 784,
      'output_layer': 10,
      'batch_size':64,
      'dropout_rate':0,
      'weight_decay':0.005,
      'learning_rate': 0.0001,
      'hidden_layer':[16,16,16],
      'optimiser':'mgd',
      'activation':'sigmoid',
      'initialisation':'xavier',
      'loss_function':'cross_entropy'

  }

  # Initialize a new wandb run
  wandb.init(project='CS-6910 A1', entity='shreyashgadgil007',config=config_defaults)
  wandb.run.name = 'b_s:'+str(wandb.config.batch_size)+',lr:'+ str(wandb.config.learning_rate)+'drop:'+str(wandb.config.dropout_rate)+',ep:'+str(wandb.config.epochs)+ ',opt:'+str(wandb.config.optimiser)+ ',hl:'+str(wandb.config.hidden_layer)+ ',act:'+str(wandb.config.activation)+',decay:'+str(wandb.config.weight_decay)+',init:'+str(wandb.config.initialisation)+',loss:'+str(wandb.config.loss_function)


  config = wandb.config
  learning_rate = config.learning_rate
  epochs = int(config.epochs)
  hidden_layer = config.hidden_layer
  activation = config.activation
  optimiser = config.optimiser
  input_layer = config.input_layer
  output_layer = config.output_layer
  batch_size = config.batch_size
  weight_decay = config.weight_decay
  loss_function = config.loss_function
  initialisation = config.initilisation
  dropout_rate=config.dropout_rate
  # Model training here
  sweep_network    = NeuralNet(input_layer, hidden_layer, output_layer,initialisation,activation,loss_function,dropout_rate)
  acc1,acc2,train_loss,val_loss  = sweep_network.train(X_train,Y_train,X_val,Y_val,learning_rate,epochs,optimiser,batch_size,weight_decay,WandB=True)
  # print('Acc',acc2)

import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)
wandb.agent(sweep_id, function=train1, count=150)
